In [ ]:
import pandas as pd
files = 'market'
columns1=['companyId', 'companyName', 'tickerSymbol', 'exchangeSymbol', 'ISOCode']
columns2=['pricingDate', 'exDate', 'payDate', 'recordDate', 'announcedDate']
columns3=['priceClose', 'volume', 'adjustmentFactor', 'VWAP', 'divAdjFactor', 'marketCap', 'TEV', 'sharesOutstanding']
columns4=['divFreqTypeName', 'supplementalTypeName', 'dividendTypeName', 'divAmount', 'netAmount', 'grossAmount',]

df = pd.read_parquet(f'./{files}.parquet', engine='fastparquet', columns=columns4)

quarter_size = len(df) // 4
for quarter in range(4):
    start_row = quarter * quarter_size
    end_row = (quarter + 1) * quarter_size if quarter < 3 else len(df)
    
    # Extract the quarter of the DataFrame
    df_quarter = df.iloc[start_row:end_row]

    # Process df_quarter as needed
    # For example, perform some operations, analysis, or save to a new file
    # ...

    # Optional: Save each quarter to a new file if needed
    df_quarter.to_parquet(f'./{files}_columns4_quarter_{quarter + 1}.parquet')

for quarter in range(1, 5):
    # Read files for each quarter
    df1 = pd.read_parquet(f'market_columns1_quarter_{quarter}.parquet')
    df2 = pd.read_parquet(f'market_columns2_quarter_{quarter}.parquet')
    df3 = pd.read_parquet(f'market_columns3_quarter_{quarter}.parquet')
    df4 = pd.read_parquet(f'market_columns4_quarter_{quarter}.parquet')

    # Combine horizontally
    combined_df = pd.concat([df1, df2, df3, df4], axis=1)

    # Optionally, save the combined DataFrame to a new file
    combined_df.to_parquet(f'market_combined_quarter_{quarter}.parquet')

In [13]:
quarter = [2, 3, 4]

for q in quarter:
    df = pd.read_parquet(f'./_SNP_dataset/market_combined_quarter_{q}.parquet', engine='fastparquet')
    # exchangeSymbol로 구분해서 exchangeSymbol별로 parquet 저장
    exchangeSymbol = df['exchangeSymbol'].unique()
    for i in exchangeSymbol:
        df_ex = df[df['exchangeSymbol'] == i]
        df_ex.to_parquet(f'./_SNP_dataset/exchange/market_quarter_{q}_{i}.parquet')